In [57]:
!pip install openpyxl

In [27]:
import pandas as pd
import numpy as np
import os


In [54]:
path = r'C:/Users/ASUS/Documents/PYTHON/PARCIAL 2C/1 MATRICES_PIXELES/pixeles.xlsx'

In [58]:
pixeles_df = pd.read_excel(path)

In [61]:
pixeles_array = pixeles_df.to_numpy()

In [62]:
print(pixeles_array[:5, :5])  


[[0 1 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 1 1 1 0]
 [0 0 0 0 0]]


## Convolución con Kernel 3x3

In [87]:

def apply_convolution(matrix, kernel_type='horizontal'):
    # Definición de kernels
    if kernel_type == 'horizontal':
        kernel = np.array([[-1, -1, -1], [0, 0, 0], [1, 1, 1]])
    elif kernel_type == 'vertical':
        kernel = np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]])
    else:
        raise ValueError("El tipo de kernel debe ser 'horizontal' o 'vertical'")
    
    # Matriz 
    output = np.zeros_like(matrix)

    # Realizar la convolución
    for i in range(1, matrix.shape[0] - 1):  
        for j in range(1, matrix.shape[1] - 1): 
            region = matrix[i-1:i+2, j-1:j+2]
            output[i, j] = np.sum(region * kernel) # Aplicación del Kernel 
    
    return output

In [88]:
# Función de convolución
convolved_matrix = apply_convolution(pixeles_array, kernel_type='horizontal')
print("Matriz después de la convolución:")
print(convolved_matrix)


Matriz después de la convolución:
[[ 0  0  0  0  0  0]
 [ 0 36 36 36 36  0]
 [ 0 36 36 36 36  0]
 [ 0 36 36 36 36  0]
 [ 0  0  0  0  0  0]]


## Padding 


In [66]:
def apply_padding(matrix, padding_size):
    # Matriz con ceros alrededor de la original
    padded_matrix = np.pad(matrix, pad_width=padding_size, mode='constant', constant_values=0)
    return padded_matrix


In [90]:
padded_matrix = apply_padding(pixeles_array, padding_size=1)
print("Matriz después de aplicar padding:")
print(padded_matrix)

Matriz después de aplicar padding:
[[ 0  0  0  0  0  0  0  0]
 [ 0  1  2  3  4  5  6  0]
 [ 0  7  8  9 10 11 12  0]
 [ 0 13 14 15 16 17 18  0]
 [ 0 19 20 21 22 23 24  0]
 [ 0 25 26 27 28 29 30  0]
 [ 0  0  0  0  0  0  0  0]]


## Stride

In [91]:
def apply_stride(matrix, kernel_type='horizontal', stride=2):
    if kernel_type == 'horizontal':
        kernel = np.array([[-1, -1, -1], [0, 0, 0], [1, 1, 1]])
    elif kernel_type == 'vertical':
        kernel = np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]])
    else:
        raise ValueError("El tipo de kernel debe ser 'horizontal' o 'vertical'")
    
    # Calculo de tamaño de la nueva matriz
    output_rows = (matrix.shape[0] - 2) // stride + 1  # +1 para incluir el último bloque
    output_cols = (matrix.shape[1] - 2) // stride + 1  
    output = np.zeros((output_rows, output_cols))

    # Aplicación convolución con el stride especificado
    row = col = 0
    for i in range(1, matrix.shape[0] - 1, stride):
        for j in range(1, matrix.shape[1] - 1, stride):
            region = matrix[i-1:i+2, j-1:j+2]
            output[row, col] = np.sum(region * kernel) #Aplicacion del kernel 
            col += 1
        col = 0
        row += 1
    
    return output


In [92]:
# Aplicación stride con un salto de 2
strided_matrix = apply_stride(pixeles_array, kernel_type='horizontal', stride=2)
print("Matriz después del stride:")
print(strided_matrix)


Matriz después del stride:
[[36. 36.  0.]
 [36. 36.  0.]]


## Stacking 

In [83]:
def apply_stacking(matrix, n):
    stacked_matrices = []
    for i in range(n):
        kernel_type = 'horizontal' if i % 2 == 0 else 'vertical'  # Alternar tipos de kernel
        convolved = apply_convolution(matrix, kernel_type)
        stacked_matrices.append(convolved)
    return np.array(stacked_matrices)

In [84]:
# Aplicación stacking para generar 3 mapas de características
n = 3
stacked_matrices = apply_stacking(pixeles_array, n)
print("Matrices apiladas:")
print(stacked_matrices)


Matrices apiladas:
[[[ 0  0  0  0  0  0]
  [ 0 36 36 36 36  0]
  [ 0 36 36 36 36  0]
  [ 0 36 36 36 36  0]
  [ 0  0  0  0  0  0]]

 [[ 0  0  0  0  0  0]
  [ 0  6  6  6  6  0]
  [ 0  6  6  6  6  0]
  [ 0  6  6  6  6  0]
  [ 0  0  0  0  0  0]]

 [[ 0  0  0  0  0  0]
  [ 0 36 36 36 36  0]
  [ 0 36 36 36 36  0]
  [ 0 36 36 36 36  0]
  [ 0  0  0  0  0  0]]]


 ## Max Polling

In [76]:
def apply_max_pooling(matrix, stride):
    output_rows = (matrix.shape[0] - 1) // stride + 1
    output_cols = (matrix.shape[1] - 1) // stride + 1
    pooled_matrix = np.zeros((output_rows, output_cols))

    for i in range(0, matrix.shape[0] - 1, stride):
        for j in range(0, matrix.shape[1] - 1, stride):
            region = matrix[i:i+2, j:j+2]  # Region de 2x2
            pooled_matrix[i // stride, j // stride] = np.max(region)

    return pooled_matrix

In [77]:
# Aplicación max pooling con un stride de 2
pooled_matrix = apply_max_pooling(pixeles_array, stride=2)
print("Matriz después de max pooling:")
print(pooled_matrix)


Matriz después de max pooling:
[[ 8. 10. 12.]
 [20. 22. 24.]
 [ 0.  0.  0.]]


In [71]:
print("Dimensiones de la matriz:", pixeles_array.shape)

Dimensiones de la matriz: (5, 6)
